# IMPORT THE LIBRARIES

In [6]:
#exc1
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from dateutil.parser import parse
from itertools import compress
from collections import OrderedDict
from datetime import date
import datetime
from datetime import timedelta
import re

# LOAD THE PI DATA and LABORATORY ANALYSIS RESULTS

In [7]:
#Sheet From Pi
#Step1: Load the Data for the month
df_JAN2020Pi = pd.read_excel(r'C:\Users\v.t.flores\Documents\POLISHING_PUMP_FLOW_DET.xlsx', sheet_name='PI_DATA', index_col=False)

#Drop the row[1]
df_JAN2020Pi = df_JAN2020Pi.drop(df_JAN2020Pi.index[0])

#Step2: Parse the DATETIME column
#df_JAN2020Pi['DATETIME'] = df_JAN2020Pi['DATETIME'].apply(lambda x: parse(str(x)))

df_JAN2020Pi.set_index('DATETIME', inplace=True)
df_comb = df_JAN2020Pi




# DECLARE THE FUNCTIONS

In [8]:
#fcn dfn
#used in the analysis results data. cleans up the sampling time entry
def time_cleaner(x):
    time_regex = re.compile(r'[0-8a-zA-Z:]', re.IGNORECASE| re.VERBOSE|re.DOTALL)
    timeact = time_regex.findall(x)
    newtime = ''.join(timeact[0:5])
    f_time = check_ending_char(newtime)
    f_time1 = check_starting_char(f_time)
    t = parse(f_time1)
    new_str_time = str(t.hour)+':'+str(t.minute)+':'+str(t.second)
    return new_str_time


#fcn for formatting the dates from range str
def datesplitter_start(x):
    return parse(x.split('to')[0])
def datesplitter_end(x):
    return parse(x.split('to')[1])

#------------------

def remove_no_good_data_N2Dry_Feed(df): #removes the string value and fills it with the last good value
    
    df = pd.to_numeric(df, errors='coerce')
    for x in range(0, len(df)):
        if type(df[x]) == str:
            df[x] = df[x-1]
    df.fillna(method='ffill', inplace=True)
    return df

#---------------------------
def replace_no_good_data_with_LastGoodValues(df_col):
    
    df_col = pd.to_numeric(df_col, errors='coerce')
    for x in range(0, len(df_col)):
        if type(df_col[x]== str):
            if det_no_data(df_col[x]) == 'Calculation':
                df_col[x] = np.nan

    df_col.fillna(method='ffill', inplace= True)
    return df_col
#--------------------------------------------
def replace_no_good_data_with_Zero(df_col):

    df_col = pd.to_numeric(df_col, errors='coerce')
    for x in range(0, len(df_col)):
        if type(df_col[x]) == str:
            if det_no_data(df_col[x]) == 'Calculation':
                df_col[x] = 0
    df_col.fillna(value=0, inplace=True)
    
    return df_col
#--------------------------------------------
def replace_neg_data_with_Zero(df_col):

    df_col = pd.to_numeric(df_col, errors='coerce')
    for x in range(0, len(df_col)):
        if df_col[x] < 0:
            df_col[x] = 0    
    return df_col
#--------------------------------------------


In [9]:
DF_105PU01 = df_comb

# Clean the Data Using the functions

In [10]:
df_comb['105TK05_LVL'] = replace_neg_data_with_Zero(df_comb['105TK05_LVL'])
df_comb['105TK05_TEMP'] = replace_neg_data_with_Zero(df_comb['105TK05_TEMP'])
df_comb['105PU01A_CUR'] = replace_neg_data_with_Zero(df_comb['105PU01A_CUR'])
df_comb['105PU01A_FLOW'] = replace_neg_data_with_Zero(df_comb['105PU01A_FLOW'])
df_comb['105FT01A_PDI'] = replace_neg_data_with_Zero(df_comb['105FT01A_PDI'])
df_comb['105PU01B_CUR'] = replace_neg_data_with_Zero(df_comb['105PU01B_CUR'])
df_comb['105PU01B_FLOW'] = replace_neg_data_with_Zero(df_comb['105PU01B_FLOW'])
df_comb['105FT01B_PDI'] = replace_neg_data_with_Zero(df_comb['105FT01B_PDI'])
df_comb['105PU01C_CUR'] = replace_neg_data_with_Zero(df_comb['105PU01C_CUR'])
df_comb['105PU01C_FLOW'] = replace_neg_data_with_Zero(df_comb['105PU01C_FLOW'])
df_comb['105FT01C_PDI'] = replace_neg_data_with_Zero(df_comb['105FT01C_PDI'])
df_comb['105PU01D_CUR'] = replace_neg_data_with_Zero(df_comb['105PU01D_CUR'])
df_comb['105PU01D_FLOW'] = replace_neg_data_with_Zero(df_comb['105PU01D_FLOW'])
df_comb['105FT01D_PDI'] = replace_neg_data_with_Zero(df_comb['105FT01D_PDI'])
df_comb['105PU01E_CUR'] = replace_neg_data_with_Zero(df_comb['105PU01E_CUR'])
df_comb['105PU01E_FLOW'] = replace_neg_data_with_Zero(df_comb['105PU01E_FLOW'])
df_comb['105FT01E_PDI'] = replace_neg_data_with_Zero(df_comb['105FT01E_PDI'])
df_comb['105PU01F_CUR'] = replace_neg_data_with_Zero(df_comb['105PU01F_CUR'])
df_comb['105PU01F_FLOW'] = replace_neg_data_with_Zero(df_comb['105PU01F_FLOW'])
df_comb['105FT01F_PDI'] = replace_neg_data_with_Zero(df_comb['105FT01F_PDI'])
df_comb['105PU01G_CUR'] = replace_neg_data_with_Zero(df_comb['105PU01G_FLOW'])

In [11]:
#Parameter list including 105TK05 temp and 105TK05 lvl
A_lst_DF = DF_105PU01[['105TK05_LVL', '105TK05_TEMP', '105PU01A_CUR', '105FT01A_PDI', '105PU01A_FLOW', 'SEL_A']]
B_lst_DF = DF_105PU01[['105TK05_LVL', '105TK05_TEMP', '105PU01B_CUR','105FT01B_PDI', '105PU01B_FLOW', 'SEL_B']]
C_lst_DF = DF_105PU01[['105TK05_LVL', '105TK05_TEMP', '105PU01C_CUR','105FT01C_PDI', '105PU01C_FLOW', 'SEL_C']]
D_lst_DF = DF_105PU01[['105TK05_LVL', '105TK05_TEMP', '105PU01D_CUR','105FT01D_PDI', '105PU01D_FLOW', 'SEL_D']]
E_lst_DF = DF_105PU01[['105TK05_LVL', '105TK05_TEMP', '105PU01E_CUR','105FT01E_PDI', '105PU01E_FLOW', 'SEL_E']]
F_lst_DF = DF_105PU01[['105TK05_LVL', '105TK05_TEMP', '105PU01F_CUR','105FT01F_PDI', '105PU01F_FLOW', 'SEL_F']]
G_lst_DF = DF_105PU01[['105TK05_LVL', '105TK05_TEMP', '105PU01G_CUR','105FT01F_PDI', '105PU01F_FLOW', 'SEL_F']]

In [12]:
A_lst_DF.dropna(inplace = True)
B_lst_DF.dropna(inplace = True)
C_lst_DF.dropna(inplace = True)
D_lst_DF.dropna(inplace = True)
E_lst_DF.dropna(inplace = True)
F_lst_DF.dropna(inplace = True)
G_lst_DF.dropna(inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we c

In [13]:
A_lst_DF = A_lst_DF[A_lst_DF['SEL_A'] != 'ON']
B_lst_DF = B_lst_DF[B_lst_DF['SEL_B'] != 'ON']
C_lst_DF = C_lst_DF[C_lst_DF['SEL_C'] != 'ON']
D_lst_DF = D_lst_DF[D_lst_DF['SEL_D'] != 'ON']
E_lst_DF = E_lst_DF[E_lst_DF['SEL_E'] != 'ON']
F_lst_DF = F_lst_DF[F_lst_DF['SEL_F'] != 'ON']


# PARAMETERS: CUR, PDI

In [14]:

A_lst_xset = A_lst_DF[['105PU01A_CUR','105FT01A_PDI']]
B_lst_xset = B_lst_DF[['105PU01B_CUR','105FT01B_PDI']]
C_lst_xset = C_lst_DF[['105PU01C_CUR','105FT01C_PDI']]
D_lst_xset = D_lst_DF[['105PU01D_CUR','105FT01D_PDI']]
E_lst_xset = E_lst_DF[['105PU01E_CUR','105FT01E_PDI']]
F_lst_xset = F_lst_DF[['105PU01F_CUR','105FT01F_PDI']]

col_a = A_lst_DF['105PU01A_FLOW']
col_b = B_lst_DF['105PU01B_FLOW']
col_c = C_lst_DF['105PU01C_FLOW']
col_d = D_lst_DF['105PU01D_FLOW']
col_e = E_lst_DF['105PU01E_FLOW']
col_f = F_lst_DF['105PU01F_FLOW']

In [15]:
A_lst_xset

,105PU01A_CUR,105FT01A_PDI
DATETIME,,
2020-01-01 00:00:00,66.482693,43.533333
2020-01-01 01:00:00,66.445520,47.816667
2020-01-01 02:00:00,65.698880,50.900000
2020-01-01 03:00:00,63.755893,50.183333
2020-01-01 04:00:00,27.769613,23.433333
...,...,...
2020-02-07 19:00:00,72.225293,9.900000
2020-02-07 20:00:00,74.062653,15.383333
2020-02-07 21:00:00,74.366613,20.550000


In [16]:
# IMPORT THE MACHINE LEARNING LIBRARIES
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression

# FUNCTIONS TO GENERATE THE COEFFICIENTS AND INTERCEPTS

In [17]:
def generate_lm_coefficients(df_column_toPredict, colname_lst, pumpLetter):
    X1 = colname_lst.dropna()
    y1 = df_column_toPredict.dropna()
    X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.20, random_state=0)

    lm = LinearRegression()
    lm.fit(X1_train,y1_train)

    coeff_df_FLOW_PU01 = pd.DataFrame(zip(lm.coef_, X1.columns), columns=['105PU01'+str(pumpLetter)+'_FLOW','PU01'+str(pumpLetter)+'_Param'])
    
    return coeff_df_FLOW_PU01

#--------------------------------------
def generate_lm_intercept(df_column_toPredict, colname_lst, pumpLetter):
    X1 = colname_lst.dropna()
    y1 = df_column_toPredict.dropna()
    X1_train, X1_test, y1_train, y1_test = train_test_split(X1, y1, test_size=0.20, random_state=0)

    lm = LinearRegression()
    lm.fit(X1_train,y1_train)

    coeff_df_FLOW_PU01 = pd.DataFrame(zip(lm.coef_, X1.columns), columns=['105PU01'+str(pumpLetter)+'_FLOW','PU01'+str(pumpLetter)+'_Param'])
    intercept_pumpflow = lm.intercept_
    
    return intercept_pumpflow

In [18]:
A_coefficients_df = generate_lm_coefficients(col_a, A_lst_xset, 'A')
A_intercept = generate_lm_intercept(col_a, A_lst_xset, 'A')

B_coefficients_df = generate_lm_coefficients(col_b, B_lst_xset, 'B')
B_intercept = generate_lm_intercept(col_b, B_lst_xset, 'B')

C_coefficients_df = generate_lm_coefficients(col_c, C_lst_xset, 'C')
C_intercept = generate_lm_intercept(col_c, C_lst_xset, 'C')

D_coefficients_df = generate_lm_coefficients(col_d, D_lst_xset, 'D')
D_intercept = generate_lm_intercept(col_d, D_lst_xset, 'D')

E_coefficients_df = generate_lm_coefficients(col_e, E_lst_xset, 'E')
E_intercept = generate_lm_intercept(col_e, E_lst_xset, 'E')

F_coefficients_df = generate_lm_coefficients(col_f, F_lst_xset, 'F')
F_intercept = generate_lm_intercept(col_f, F_lst_xset, 'F')

In [19]:
C_coefficients_df

,105PU01C_FLOW,PU01C_Param
0,3.801696,105PU01C_CUR
1,-0.580304,105FT01C_PDI


In [30]:
B_coefficients_df

,105PU01B_FLOW,PU01B_Param
0,3.922221,105PU01B_CUR
1,-0.550560,105FT01B_PDI


In [29]:
A_coefficients_df.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\A_coeff.xlsx')
B_coefficients_df.to_ex(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\A_coeff.xlsx')

TypeError: 'DataFrame' object is not callable

In [31]:
A_coefficients_df.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\A_coeff.xlsx')

B_coefficients_df.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\B_coeff.xlsx')

C_coefficients_df.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\C_coeff.xlsx')

D_coefficients_df.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\D_coeff.xlsx')

E_coefficients_df.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\E_coeff.xlsx')

F_coefficients_df.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\F_coeff.xlsx')


In [32]:
A_intercept.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\A_intercept.xlsx')
B_intercept.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\B_intercept.xlsx')
C_intercept.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\C_intercept.xlsx')
D_intercept.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\D_intercept.xlsx')
E_intercept.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\E_intercept.xlsx')
F_intercept.to_excel(r'\\THGOBNA002\thpal\05 Production\150 DCS\00 DCS Monitoring & Records\00 Active Files\2020\Polishing_Filter_Flow_Pred\F_intercept.xlsx')

AttributeError: 'numpy.float64' object has no attribute 'to_excel'

In [34]:
print(A_intercept,
B_intercept,
C_intercept,
D_intercept,
E_intercept,
F_intercept)

685.8273879919026 -9.212248576009785 -5.421782466672454 -10.387774950324456 -6.4146067805576195 -10.276708430233413
